In [431]:
%pip install pandas
import pandas as pd


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [432]:
stats = pd.read_csv("teamCsvFiles/OaklandAthletics.csv", encoding='latin1')
stats.fillna(0, inplace=True)
teamStats = None

In [433]:
stats.columns

Index(['gameId', 'teamId', 'opponentId', 'homeTeam?', 'playerInfoId',
       'playerFullName', 'battingRuns', 'battingDoubles', 'battingTriples',
       'battingHomeRuns', 'battingStrikeOuts', 'battingBaseOnBalls',
       'battingHits', 'battingAtBats', 'battingStolenBases', 'battingRbi',
       'battingLeftOnBase', 'pitchingRuns', 'pitchingDoubles',
       'pitchingTriples', 'pitchingHomeRuns', 'pitchingStrikeOuts',
       'pitchingBaseOnBalls', 'pitchingHits', 'pitchingAtBats',
       'pitchingStolenBases', 'pitchingNumberOfPitches',
       'pitchingInningsPitched', 'pitchingWins', 'pitchingLosses',
       'pitchingHolds', 'pitchingBlownSaves', 'pitchingEarnedRuns',
       'pitchingPitchesThrown'],
      dtype='object')

In [434]:
def formatPlayerStats(playerdf):
    game_scores = playerdf.groupby('gameId').agg(
    teamScore=('battingRuns', 'sum'),
    opponentScore=('pitchingRuns', 'sum')
)
    # Assign 'Win' or 'Loss' based on the comparison of scores
    game_scores['result'] = game_scores.apply(lambda row: 'Win' if row['teamScore'] > row['opponentScore'] else 'Loss', axis=1)
    # Merge the result back to the original dataframe based on gameId
    playerdf = playerdf.merge(game_scores['result'], left_on='gameId', right_index=True)
    playerdf["battingAvg"] = (playerdf["battingHits"]) / (stats["battingAtBats"])
    playerdf["onBasePercentage"] = (playerdf["battingHits"] + playerdf["battingBaseOnBalls"]) / (playerdf["battingAtBats"] + playerdf["battingBaseOnBalls"])
    playerdf.fillna(0, inplace=True) 
    return playerdf

In [442]:
def formatTeamStats(playerdf):
    aggdf = playerdf.groupby(['playerFullName']).sum().reset_index()
    # Drop the 'teamId' column
    aggdf = aggdf.drop(['teamId','gameId','opponentId','homeTeam?','playerInfoId'], axis=1)  
    aggdf["battingAvg"] = (aggdf["battingHits"]) / (aggdf["battingAtBats"])
    aggdf["onBasePercentage"] = (aggdf["battingHits"] + aggdf["battingBaseOnBalls"]) / (aggdf["battingAtBats"] + aggdf["battingBaseOnBalls"])
    aggdf.fillna(0, inplace=True)
    return aggdf

In [451]:
stats = formatPlayerStats(stats)

In [453]:
teamStats = formatTeamStats(stats)
teamStats = teamStats.drop(["result"],axis=1)